In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df=pd.read_csv("/content/train.csv")
df.shape
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)    #pura value aa jye ga
df.info()
df.isnull().sum()
plt.figure(figsize=(25,25))
sns.heatmap(df.isnull())
null_var=df.isnull().sum()/df.shape[0]*100  # null value ka % mile ga
null_var
drop_columns =null_var[null_var>20].keys() # condition lga diya h jisme 17 se upr null h
drop_columns

df2_drop_clm=df.drop(columns=drop_columns)

df2_drop_clm.shape

sns.heatmap(df2_drop_clm.isnull())

df3_drop_rows=df2_drop_clm.dropna()

df3_drop_rows.shape

df3_drop_rows.isnull().sum()

df3_drop_rows.isnull().sum()

df3_num=df2_drop_clm.select_dtypes(include=['int64','float64'])
df3_num

sns.heatmap(df3_num)

df3_num[df3_num.isnull().any(axis=1)]

df3_num.isnull().sum()

missing_num_var=[var for var in df3_num.columns if df3_num[var].isnull().sum()>0]
missing_num_var

df4_num_mean=df3_num.fillna(df3_num.mean())

df4_num_mean.isnull().sum().sum()

df5_num_median=df3_num.fillna(df3_num.median())
df5_num_median.isnull().sum().sum()

FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error

# ---------------------------------------
# 1. Load Data
# ---------------------------------------
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")     # optional

TARGET = "target"   


# ---------------------------------------
# 2. Split X and y
# ---------------------------------------
X = train.drop(columns=[TARGET])
y = train[TARGET]


# Detect Classification vs Regression
is_classification = (y.dtype == "object") or (y.nunique() <= 20)
print("Problem Type:", "Classification" if is_classification else "Regression")


# ---------------------------------------
# 3. Separate numeric & categorical columns
# ---------------------------------------
num_cols = X.select_dtypes(include=['int64','float64']).columns
cat_cols = X.select_dtypes(include=['object','category']).columns


# ---------------------------------------
# 4. MANUAL Preprocessing
# ---------------------------------------

# ---- Numeric: fill missing values with median ----
num_imputer = SimpleImputer(strategy='median')
X_num = num_imputer.fit_transform(X[num_cols])
test_num = num_imputer.transform(test[num_cols])

# ---- Categorical: fill missing + one-hot encode ----
cat_imputer = SimpleImputer(strategy='constant', fill_value="__MISSING__")
X_cat_filled = cat_imputer.fit_transform(X[cat_cols])
test_cat_filled = cat_imputer.transform(test[cat_cols])

ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_cat = ohe.fit_transform(X_cat_filled)
test_cat = ohe.transform(test_cat_filled)

# ---- Combine numeric + categorical ----
X_final = np.hstack([X_num, X_cat])
test_final = np.hstack([test_num, test_cat])


# ---------------------------------------
# 5. Train / Validation Split
# ---------------------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X_final, y, test_size=0.2, random_state=42,
    stratify=y if is_classification else None
)


# ---------------------------------------
# 6. Train the Model
# ---------------------------------------
if is_classification:
    model = RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    )
else:
    model = RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    )

model.fit(X_train, y_train)


# ---------------------------------------
# 7. Evaluate
# ---------------------------------------
y_pred = model.predict(X_val)

if is_classification:
    try:
        y_proba = model.predict_proba(X_val)[:,1]
        print("ROC AUC:", roc_auc_score(y_val, y_proba))
    except:
        print("Accuracy:", accuracy_score(y_val, y_pred))
else:
    print("RMSE:", mean_squared_error(y_val, y_pred, squared=False))


# ---------------------------------------
# 8. Predict Test + Create Submission
# ---------------------------------------
pred_test = model.predict(test_final)

if "id" in test.columns:
    submission = pd.DataFrame({
        "id": test["id"],
        TARGET: pred_test
    })
else:
    submission = pd.DataFrame({
        TARGET: pred_test
    })

submission.to_csv("submit.csv", index=False)
print("submit.csv ")